# Neural Creative Transfer

We implement an modification of Gatys, Ecker and Bethge's [neural style transfer algorithm](http://arxiv.org/abs/1508.06576) for our creative assets.

The algorith uses a VGG network to separate and recombine content of an creative and style of a website, and therefore provides a system for creating native creative for advertising. 